#### Image Crop
* 이미지 전처리 및 컨투어
* Green, Red 처리 이미지로 세포 좌표 받아와 원본 이미지에서 크롭함

In [1]:
import numpy as np
import cv2, sys
from matplotlib import pyplot as plt
import glob
from PIL import Image

In [ ]:
image_num=4

### Test dataset 생성(221017)

In [2]:
# 파일이름만 바꿔주기
# Test용
#image_num=1

for i in range(1,30):
    image_num=i

    image=cv2.imread(f'./2210_Retry/june/Test/{image_num}.tif')

    gray_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

    blurI=cv2.GaussianBlur(gray_image, ksize=(7,7), sigmaX=0)

    ret, dst=cv2.threshold(blurI, 127, 255, cv2.THRESH_BINARY)

    kernel=np.ones((100,100),np.uint8)

    dilation_edgedI=cv2.dilate(dst,kernel,iterations=1)

    blurI=cv2.GaussianBlur(dilation_edgedI, ksize=(7,7), sigmaX=0)
    ret, thresh1=cv2.threshold(blurI,127,255,cv2.THRESH_BINARY)
    # -> gaussian blur 처리 하면 모두 지워짐을 확인

    normR=cv2.normalize(blurI,None,0,255,cv2.NORM_MINMAX)

    edgedI=cv2.Canny(normR,10,30)

    kernel=np.ones((10,10),np.uint8)

    dilation_edged2_I=cv2.dilate(edgedI,kernel,iterations=1)

    kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
    closed_I=cv2.morphologyEx(dilation_edged2_I,cv2.MORPH_CLOSE,kernel)

    contours, _=cv2.findContours(closed_I.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    total=0

    contours_xy=np.array(contours,dtype=object,)
    contours_xy.shape

    count=0 #contour 처리한 총 이미지 개수를 알기 위함
    for c in contours:
        x,y,w,h=cv2.boundingRect(c)

        #print(x,y,w,h)
        #print(w*h)
        result = w*h

        #if(result>50830): 또는 if(w>100 and h>100): (전자는 filtering 거친 경우, 후자는 Exception label 쓰는 경우)
        if(result>50830):
            img_trim = image[y:y+h, x:x+w]
            #cv2.imwrite(f'{count}.jpg', img_trim)
            cv2.imwrite(f'./2210_Retry/june/Test/ModelTest/{image_num}_{count}.jpg',img_trim)
            count+=1
            org_image = cv2.imread('org_trim.jpg')
    #print(count)


### Train dataset 생성(221017)

In [2]:
# 파일이름만 바꿔주기
# Train용으로 되어있음
# 필터링 유무에 따라 result=w*h 부분 주석처리 확인하기
#image_num=1

for i in range(76,106):
    image_num=i
    image=cv2.imread(f'./2210_Retry/june/Train/vscode_optical/{image_num}.tif')
    image_red=cv2.imread(f'./2210_Retry/june/Train/vscode_red/{image_num}.tif')
    image_green=cv2.imread(f'./2210_Retry/june/Train/vscode_green/{image_num}.tif')

    #gray_red=cv2.cvtColor(image_red,cv2.COLOR_BGR2GRAY)
    #gray_green=cv2.cvtColor(image_green,cv2.COLOR_BGR2GRAY)

    ret, dst=cv2.threshold(image_red, 127, 255, cv2.THRESH_BINARY)
    ret, dst2=cv2.threshold(image_green,127, 255, cv2.THRESH_BINARY) #추가된 부분

    kernel=np.ones((150,150),np.uint8)

    dilation_edgedR=cv2.dilate(dst,kernel,iterations=1)
    dilation_edgedG=cv2.dilate(dst2,kernel,iterations=1) #추가된 부분

    #cv2.imshow('Dilation Edged Red',dilation_edgedR)
    #cv2.waitKey()
    #cv2.destroyAllWindows()
    #cv2.waitKey(1)


    # Red 세포 이미지 대해서 작업
    blurR=cv2.GaussianBlur(dilation_edgedR, ksize=(7,7), sigmaX=0)
    ret, thresh1=cv2.threshold(blurR,127,255,cv2.THRESH_BINARY)

    normR=cv2.normalize(blurR,None,0,255,cv2.NORM_MINMAX)
    edgedR=cv2.Canny(normR,10,30)

    # Green 세포 이미지 대해서 작업
    blurG=cv2.GaussianBlur(dilation_edgedG, ksize=(7,7), sigmaX=0)
    ret, thresh2=cv2.threshold(blurG,127,255,cv2.THRESH_BINARY)

    normG=cv2.normalize(blurG,None,0,255,cv2.NORM_MINMAX)
    edgedG=cv2.Canny(normG,10,30)


    kernel=np.ones((50,50),np.uint8)

    dilation_edged2_R=cv2.dilate(edgedR,kernel,iterations=1)
    dilation_edged2_G=cv2.dilate(edgedG,kernel,iterations=1)

    kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))

    closed_R=cv2.morphologyEx(dilation_edged2_R,cv2.MORPH_CLOSE,kernel)
    closed_G=cv2.morphologyEx(dilation_edged2_G,cv2.MORPH_CLOSE,kernel)


    contours, _=cv2.findContours(closed_R.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    total=0

    contours_xy=np.array(contours,dtype=object,)
    contours_xy.shape

    count=0 #contour 처리한 총 이미지 개수를 알기 위함
    for c in contours:
        x,y,w,h=cv2.boundingRect(c)

        #print(x,y,w,h)
        #print(w*h)
        result=w*h

        if(w>100 and h>100):
            img_trim = image[y:y+h, x:x+w]
            #cv2.imwrite(f'{count}.jpg', img_trim)
            cv2.imwrite(f'./2210_Retry/june/Train/Red/{image_num}_{count}.jpg',img_trim)
            count+=1
            org_image = cv2.imread('org_trim.jpg')
    print(count)


    contoursG, _=cv2.findContours(closed_G.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    total=0

    contours_xy=np.array(contoursG,dtype=object,)
    contours_xy.shape

    count=0 #contour 처리한 총 이미지 개수를 알기 위함
    for c in contoursG:
        x,y,w,h=cv2.boundingRect(c) #주어진 점을 감싸는 최소 크기의 bounding box 반환

        #print(x,y,w,h)
        #print(w*h)
        result=w*h

        #if(result>50830):
        if(w>100 and h>100):
            img_trim = image[y:y+h, x:x+w]
            #cv2.imwrite(f'{count}.jpg', img_trim)
            cv2.imwrite(f'./2210_Retry/june/Train/Green/{image_num}_{count}.jpg',img_trim)
            count+=1
            org_image = cv2.imread('org_trim.jpg')
    #print(count)

1
2
7
3
5
0
1
1
1
0
2
1
0
1
0
0
0
4
1
0
0
1
0
0
0
2
0
0
1
0


#### Image Augmentation 하기
* 90, 180, 270도 뒤집어서, 1개의 이미지당 총 4개의 이미지로 생성되도록

In [ ]:
image_files_trainG = glob.glob('./2210_Retry/june/Train/Green/*.jpg')
image_files_trainR = glob.glob('./2210_Retry/june/Train/Red/*.jpg')
image_files_trainE = glob.glob('./2210_Retry/june/Train/Exception/*.jpg')

count=0

for i in image_files_trainG:
    temp_image = Image.open(i)
    temp_image_array=np.asarray(temp_image)

    #https://tempdev.tistory.com/32 (참고)
    temp_image2=cv2.cvtColor(temp_image_array,cv2.COLOR_BGR2RGB)

    temp_image90 = cv2.rotate(temp_image2,cv2.ROTATE_90_CLOCKWISE)
    temp_image180 = cv2.rotate(temp_image2,cv2.ROTATE_180)
    temp_image270 = cv2.rotate(temp_image2,cv2.ROTATE_90_COUNTERCLOCKWISE)

    cv2.imwrite(f'./2210_Retry/june/Train/augmentation_result/Green/G90_{count}.jpg',temp_image90)
    cv2.imwrite(f'./2210_Retry/june/Train/augmentation_result/Green/G180_{count}.jpg',temp_image180)
    cv2.imwrite(f'./2210_Retry/june/Train/augmentation_result/Green/G270_{count}.jpg',temp_image270)

    count+=1

count=0

for i in image_files_trainR:
    temp_image = Image.open(i)
    temp_image_array=np.asarray(temp_image)

    #https://tempdev.tistory.com/32 (참고)
    temp_image2=cv2.cvtColor(temp_image_array,cv2.COLOR_BGR2RGB)

    temp_image90 = cv2.rotate(temp_image2,cv2.ROTATE_90_CLOCKWISE)
    temp_image180 = cv2.rotate(temp_image2,cv2.ROTATE_180)
    temp_image270 = cv2.rotate(temp_image2,cv2.ROTATE_90_COUNTERCLOCKWISE)

    cv2.imwrite(f'./2210_Retry/june/Train/augmentation_result/Red/R90_{count}.jpg',temp_image90)
    cv2.imwrite(f'./2210_Retry/june/Train/augmentation_result/Red/R180_{count}.jpg',temp_image180)
    cv2.imwrite(f'./2210_Retry/june/Train/augmentation_result/Red/R270_{count}.jpg',temp_image270)

    count+=1


count=0

for i in image_files_trainE:
    temp_image = Image.open(i)
    temp_image_array=np.asarray(temp_image)

    #https://tempdev.tistory.com/32 (참고)
    temp_image2=cv2.cvtColor(temp_image_array,cv2.COLOR_BGR2RGB)

    temp_image90 = cv2.rotate(temp_image2,cv2.ROTATE_90_CLOCKWISE)
    temp_image180 = cv2.rotate(temp_image2,cv2.ROTATE_180)
    temp_image270 = cv2.rotate(temp_image2,cv2.ROTATE_90_COUNTERCLOCKWISE)

    cv2.imwrite(f'./2210_Retry/june/Train/augmentation_result/Exception/E90_{count}.jpg',temp_image90)
    cv2.imwrite(f'./2210_Retry/june/Train/augmentation_result/Exception/E180_{count}.jpg',temp_image180)
    cv2.imwrite(f'./2210_Retry/june/Train/augmentation_result/Exception/E270_{count}.jpg',temp_image270)

    count+=1

In [ ]:
for i in range(1,61,3):
    print(i)